In [1]:
from simulator import blogm, bSqc, Neg, Sa
import torch
from math import prod
from functools import reduce
import pandas
from utils import dtype, device, pauli, basis, torch_data

In [2]:
seed = 1
train_ratio = 8/9
all_N = []
all_I = []
all_S = []
for N in range(4,36,2):
    torch.manual_seed(seed)
    prepseq, shadow_state, rhoS = torch_data(f'data/data_{N}na.pickle', shuffle=True)
    train_size = int(prepseq.shape[0]*train_ratio)
    test_size = prepseq.shape[0]-train_size
    
    prepseq_train, prepseq_test = prepseq[:train_size], prepseq[train_size:]
    shadow_state_train, shadow_state_test = shadow_state[:train_size], shadow_state[train_size:]
    rhoS_train, rhoS_test = rhoS[:train_size], rhoS[train_size:]
    N_value = []
    I_value = []
    S_value = []
    for i in range(2):
        for j in range(2):
            train_idx = (prepseq_train[:,::2].sum(-1)%2 == i)*(prepseq_train[:,1::2].sum(-1)%2 == j)
            test_idx = (prepseq_test[:,::2].sum(-1)%2 == i)*(prepseq_test[:,1::2].sum(-1)%2 == j)
            rS = rhoS_test[test_idx]
            rC = rhoS_train[train_idx].mean(0)
            # e = torch.linalg.eigh(rC)[0].min()
            # rC = rC + torch.eye(4,4)*(e.abs()-e)/2
            # rC = rC/torch.trace(rC)
            rC = rC.unsqueeze(0).expand(rS.shape[0],-1,-1)
            N_value.append(Neg(rS, rC))
            S_value.append(bSqc(rS, rC))
            I_value.append(Sa(rS, rC)-bSqc(rS, rC))
    all_N.append(torch.cat(N_value))
    all_I.append(torch.cat(I_value))
    all_S.append(torch.cat(S_value))
    print(f'N={N} done')
torch.save(torch.cat(all_N).view(16,-1), f'../1d-result-figure/ps_N_values.pt')
torch.save(torch.cat(all_S).view(16,-1), f'../1d-result-figure/ps_S_values.pt')
torch.save(torch.cat(all_I).view(16,-1), f'../1d-result-figure/ps_I_values.pt')


N=4 done
N=6 done
N=8 done
N=10 done
N=12 done
N=14 done
N=16 done
N=18 done
N=20 done
N=22 done
N=24 done
N=26 done
N=28 done
N=30 done
N=32 done
N=34 done


In [5]:
torch.cat(all_I).view(16,-1).mean(-1)

tensor([ 0.3288,  0.2772,  0.1728,  0.0468, -0.0443, -0.0638, -0.1326, -0.1644,
        -0.2326, -0.2727, -0.3386, -0.3676, -0.4057, -0.4158, -0.4548, -0.4669],
       dtype=torch.float64)